In [1]:
from pathlib import Path
import pandas as pd
from sklearn.model_selection import StratifiedKFold

In [2]:
ROOT = "/workspace/data/typhoon/train"
N_SPLITS = 5
DOWN_SAMPLING_N = 57423

In [3]:
def create_kfold_dataframe(df: pd.DataFrame, n_splits=5, random_state=0):
    # KFoldを設定
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    # fold列を初期化
    df['fold'] = -1

    X, y = df["image_path"], df["label"]

    for fold_number, (train_index, valid_index) in enumerate(skf.split(X, y)):
        df.loc[valid_index, 'fold'] = fold_number
    
    return df

In [4]:
def downsample_nonTC(df: pd.DataFrame, N: int, random_state: int = 42) -> pd.DataFrame:
    """
    データフレームを処理し、nonTCデータをダウンサンプリングする関数

    Parameters:
    df (pandas.DataFrame): 処理するデータフレーム
    N (int): 各foldでnonTCをダウンサンプリングする数

    Returns:
    pandas.DataFrame: 処理後のデータフレーム
    """
    # TCデータを抽出
    df_TC = df[df['label'] == 'TC']

    # nonTCデータを抽出
    df_nonTC = df[df['label'] == 'nonTC']

    # foldごとにnonTCデータをダウンサンプリング
    downsampled_nonTC_dfs = []
    for fold in range(5):  # 0から4までのfold
        fold_df = df_nonTC[df_nonTC['fold'] == fold]
        if len(fold_df) > N:
            downsampled_fold_df = fold_df.sample(n=N, random_state=42)
        else:
            downsampled_fold_df = fold_df
        downsampled_nonTC_dfs.append(downsampled_fold_df)

    # ダウンサンプリングしたnonTCデータを結合
    df_nonTC_downsampled = pd.concat(downsampled_nonTC_dfs, ignore_index=True)

    # TCデータとダウンサンプリングしたnonTCデータを結合
    downsampled_df = pd.concat([df_TC, df_nonTC_downsampled], ignore_index=True)

    return downsampled_df

In [5]:
# ルートディレクトリを指定
root_directory = Path(ROOT)

In [6]:
# tifファイルのパスを再帰的に取得
tif_files = list(root_directory.rglob("*.tif"))

In [7]:
paths, labels, dirs, filenames = [], [], [], []
for file_path in tif_files:
    paths.append(str(file_path))
    labels.append(str(file_path.parent.name))
    dirs.append(str(file_path.parent.parent.name))
    filenames.append(str(file_path.name))

In [8]:
df = pd.DataFrame({
    "image_path": paths,
    "subdir": dirs,
    "file_name": filenames,
    "label": labels,
})

In [9]:
# K分割データフレームを作成
df = create_kfold_dataframe(df, N_SPLITS)

In [10]:
downsampled_df = downsample_nonTC(df, N=DOWN_SAMPLING_N)

In [11]:
downsampled_df.to_csv('../inputs/train_val_kfold_split_downsampling.csv', index = False)

In [12]:
# 結果の確認
print("処理後のデータフレームの形状:", downsampled_df.shape)
print("TCデータの数:", len(downsampled_df[downsampled_df['label'] == 'TC']))
print("nonTCデータの数:", len(downsampled_df[downsampled_df['label'] == 'nonTC']))
print("foldごとのnonTCデータの数:")
print(downsampled_df[downsampled_df['label'] == 'nonTC']['fold'].value_counts().sort_index())

処理後のデータフレームの形状: (358894, 5)
TCデータの数: 71779
nonTCデータの数: 287115
foldごとのnonTCデータの数:
fold
0    57423
1    57423
2    57423
3    57423
4    57423
Name: count, dtype: int64
